In [ ]:
# pip install llama-index
# pip install --upgrade pydantic==1.10.0 typing-extensions==4.5.0

# Problem Statement

#### We want to perform RAG on the retrieved results from SQL DB.

In [1]:
from sqlalchemy import (
    create_engine,
    text,
)

import pandas as pd
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
from llama_index import Document, ListIndex
from llama_index import SQLDatabase, ServiceContext
from llama_index.llms import ChatMessage, OpenAI
from typing import List
import ast
import openai
from IPython.display import display, HTML

# Use existing product reviews

In [2]:
engine = create_engine("sqlite:///TrendyolProduct.sqlite3")

sql_database = SQLDatabase(engine, include_tables=["TBL_Product", "TBL_Comment"])
conn = sql_database.engine.connect()

In [3]:
with engine.begin() as conn:
    query = text("""SELECT * FROM TBL_Product""")
    df_product = pd.read_sql_query(query, conn)
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4109 entries, 0 to 4108
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Product_Id     4109 non-null   int64 
 1   Product_Name   4109 non-null   object
 2   Product_Brand  4109 non-null   object
 3   Product_Link   4109 non-null   object
dtypes: int64(1), object(3)
memory usage: 128.5+ KB


In [4]:
with engine.begin() as conn:
    query = text("""SELECT * FROM TBL_Comment""")
    df_comment = pd.read_sql_query(query, conn)
df_comment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43923 entries, 0 to 43922
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Comment_Id          43923 non-null  int64 
 1   Product_Id          43923 non-null  int64 
 2   Comment_Content     43923 non-null  object
 3   Comment_Evaluation  43923 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.3+ MB


In [5]:
df_merge = pd.merge(df_product, df_comment, on="Product_Id")
df_merge.head()

,Product_Id,Product_Name,Product_Brand,Product_Link,Comment_Id,Comment_Content,Comment_Evaluation
0,1,Erkek %100 Pamuk Koyu Vizon Polo Yaka Düz T-sh...,Avva,https://www.trendyol.com/avva/erkek-100-pamuk-...,1,Babam için almıştım aşırııı güzell,5
1,1,Erkek %100 Pamuk Koyu Vizon Polo Yaka Düz T-sh...,Avva,https://www.trendyol.com/avva/erkek-100-pamuk-...,2,Dokusu çok güzel kalitesi çok iyi,5
2,1,Erkek %100 Pamuk Koyu Vizon Polo Yaka Düz T-sh...,Avva,https://www.trendyol.com/avva/erkek-100-pamuk-...,3,renk ve kalite çok güzel AVVA sonuçta indirimd...,5
3,1,Erkek %100 Pamuk Koyu Vizon Polo Yaka Düz T-sh...,Avva,https://www.trendyol.com/avva/erkek-100-pamuk-...,4,Çok beğendim güzel,5
4,1,Erkek %100 Pamuk Koyu Vizon Polo Yaka Düz T-sh...,Avva,https://www.trendyol.com/avva/erkek-100-pamuk-...,5,Güzel tişört,5


In [18]:
df_merge.Product_Brand.value_counts()

Koton           11576
Mavi            11350
Defacto          8679
US Polo Assn     4149
Avva             3181
Suwen            2426
HUMMEL            912
Nike              515
Colin’s           287
adidas            255
Puma              236
Tudors            210
Derimod            56
Loft               55
LC Waikiki         36
Name: Product_Brand, dtype: int64

# SQL + RAG

**Three-Step Process to Answer Complex Questions using SQL and List Indexing:**

1. **Decomposition of the Question:**

*   **Primary Query Formation:** Phrase the main question in natural language suitable for extracting preliminary data from the SQL table.
*   **Secondary Query Formation:** Formulate an auxiliary question that operates on the results of the primary query. This will guide the final extraction

2. **Data Retrieval:** Execute the primary SQL query to gather the initial set of results.

3. **Final Answer Generation:** Utilize List Index to refine or interpret the results based on the secondary question, leading to the final desired answer.

# Decomposition of the Question

In [6]:
# set llm
openai.api_key = 'sk-...'

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

In [7]:
def generate_questions(user_query: str) -> List[str]:
  system_message = '''
  Sen Türkçe dilinde geliştirildin ve verdiğin cevaplar Türkçe olmalı. Sana aşağıdaki sütunları içeren Sqlite tablosu verilmektedir.

   Product_Name, Product_Brand, Comment_Content, Comment_Evaluation.

   Görevin verilen soruyu aşağıdaki iki soruya ayrıştırmaktır.

   1. Tablodan sonuç almak için sorulması gereken doğal dilde soru.
   2. Son cevabın sağlanması için ilk sorudan çıkan sonuca istinaden sorulması gereken soru.

   Örnek:

   Girdi:
   Nüfusu 5000000'den fazla olan ülkelerin kültürü nasıldır?

   Çıktı:
   1. Nüfusu 5000000'den fazla olan ülkelerin yorumlarını alın
   2. Ülkelerin kültürünü sağlayın
  '''

  messages = [
      ChatMessage(role="system", content=system_message),
      ChatMessage(role="user", content=user_query),
  ]
  generated_questions = llm.chat(messages).message.content.split('\n')

  return generated_questions

In [8]:
user_query = "Markası Avva olan ürünlerin yorumları nasıldır?"

text_to_sql_query, rag_query = generate_questions(user_query)
print(text_to_sql_query)
print("*"*50)
print(rag_query)

1. Markası Avva olan ürünlerin yorumlarını alın.
**************************************************
2. Yorumların nasıl olduğunu sağlayın.


# Execute the primary SQL query to gather the initial set of results.

In [9]:
# Create SQL Query Engine
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["TBL_Product", "TBL_Comment"],
    synthesize_response = False,
    service_context = service_context
)

In [10]:
import random

In [11]:
sql_response = sql_query_engine.query(text_to_sql_query)

sql_response_list = ast.literal_eval(sql_response.response)

# shuffle the list
random.shuffle(sql_response_list)

text = [' '.join(t) for t in sql_response_list[:20]] # get first 20 rows
text = '\n'.join(text)

In [12]:
print(text)

avva kaliteai
Ürün çok güzel ama bir yıldızı geç gelmesinden dolayı kırdım sipariş verdikten 1 hafta sonra geldi neredeyse
5te5 bir ürün
Ürün için pamuklu deniyor fakat naylon gibi daha çok.
siyahinida aldım gayet guzel
kaliteli...
Çok güzel bir ürün kumaşı duruşu yani herşeyi çok ama çok güzel kesinlikle alınmalı
HARİKA, HAYATIMA BİR PANTOLON OLARAK DEVAM ETSEM KESINLIKLE BU PANTOLON OLURDUM. DÜŞÜNMEDEN ALIN TÜM RENKLERİNİ SİPARİŞ VERİYORUM.
begendim, güzel.
bel olarak bedenime uygun bacak kısımlarında terzide işlem yaptırıp daralttirdim
Beğenerek kullanıyor babam
tam yazlık çok güzel
İndirimde aldık çok kaliteli
Eşim çok beğendi
l beden normal boyuttan büyük ona göre alın
Çok güzel pantolon 90 kg 33 beden tam oturdu, 1 tane daha sipariş verme fikri aklım çık lütfen
çok şık ve kaliteli
Kardeşim 180 boya 76 kilo ve 33 beden aldık tam oldu. Pantolon çok kaliteli kesinlikle alın
Sözelci Anne ♥️ tavsiyesi ile eşime aldım, indirimi haber verdiği için çok teşekkür ederim. Ürün çok güzel ve 

In [13]:
# Generated SQL Query
display(HTML(f'<p style="font-size:20px">{sql_response.metadata["sql_query"]}</p>'))

# List Index to refine or interpret the results based on the secondary question

In [14]:
listindex = ListIndex([Document(text=text)])
list_query_engine = listindex.as_query_engine()

In [15]:
summary = list_query_engine.query(rag_query)

In [16]:
# Final answer
display(HTML(f'<p style="font-size:20px">{summary.response}</p>'))

# Now let's wrap everything under a function.

In [24]:
def sql_rag(user_query: str) -> str:
  text_to_sql_query, rag_query = generate_questions(user_query)
  print(text_to_sql_query)
  print("*"*50)
  print(rag_query)

  sql_response = sql_query_engine.query(text_to_sql_query)

  print(sql_response.metadata["sql_query"])
  

  # shuffle the list
  random.shuffle(sql_response_list)

  text = [' '.join(t) for t in sql_response_list[:50]] # get first 50 rows
  text = '\n'.join(text)

  listindex = ListIndex([Document(text=text)])
  list_query_engine = listindex.as_query_engine()

  summary = list_query_engine.query(rag_query)

  return summary.response

# Let's try out few examples.

In [30]:
sql_rag("'Nike' marka yorumlarırnda, ayakkabı rahatlığı nasıl?")

1. 'Nike' marka ürünlere ait yorumları alın.
**************************************************
2. Ayakkabı rahatlığı hakkında yorumları sağlayın.
SELECT Comment_Content 
FROM TBL_Comment 
JOIN TBL_Product ON TBL_Comment.Product_Id = TBL_Product.Product_Id 
WHERE TBL_Product.Product_Brand = 'Nike'


'Çok güzel ve kaliteli bir ürün.'

In [28]:
sql_rag("'Koton' yorumlarını özetler misin?")

1. Koton ürünlerinin yorumlarını alın.
**************************************************
2. Koton ürünlerinin genel olarak nasıl değerlendirildiğini sağlayın.
SELECT Comment_Content, Comment_Evaluation
FROM TBL_Comment
JOIN TBL_Product ON TBL_Comment.Product_Id = TBL_Product.Product_Id
WHERE TBL_Product.Product_Brand = 'Koton'


'Koton ürünlerinin genel olarak olumlu değerlendirildiği görülmektedir. Müşteriler, ürünlerin kaliteli olduğunu ve güzel göründüğünü belirtmektedir. Ayrıca, ürünlerin rahat olduğu ve yazlık kullanıma uygun olduğu da vurgulanmaktadır. Ancak bazı müşteriler, ürünlerin beden konusunda dikkat edilmesi gerektiğini ve bazı durumlarda kesimin slim fit gibi olduğunu ifade etmektedir.'